In [4]:
# Author: Dung Ha - May 15th 2024# Install the gurobi optimizer on your system as well as import all the needed packages. Then run the cells in order.
# The first cell contains the representations code.
# The second cell contains the model code.
# The third cell converts model output in to graphical and textual representations.
# To run on other systems than Windows, import the correct packages for that system and make appropriate changes to the code for importing packages.

import tkinter as tk
from tkinter import ttk
from tkinter import Button
import time
from PIL import ImageGrab

# -------------------------- Documentation in the cell below ------------------------------------
last_course1 = 5
last_course1_Fixed = 2
last_course2 = 14     # 10
last_course2_Fixed = last_course1 + 3
last_course3 = last_course2+1     # lab
last_course4 = last_course3+1 #12     # common hour 
last_course5 = last_course4+3 #15     # lang morning
last_course6 = last_course5+1 #16     # lang afternoon
last_course7 = last_course6+1 #17      # commonhr 1h
last_course8 = last_course7+1 #18     # faculty meeting
last_course9 = last_course8+1 #19     # seminar
num_courses = last_course9 +1
num_periods = 16

courses = range(num_courses) 
course1 = range(last_course1 + 1)
course2 = range(last_course1 + 1, last_course2 + 1)
course3 = range(last_course2 +1 , last_course3 + 1)
course4 = range(last_course3 +1, last_course4 + 1)
course5 = range(last_course4 +1, last_course5 + 1)
course6 = range(last_course5 + 1, last_course6 +1)
course7 = range(last_course6 + 1, last_course7 +1)
course8 = range(last_course7 + 1, last_course8 +1)
course9 = range(last_course8 + 1, last_course9 +1)
course10 = range(last_course1_Fixed + 1)
course11 = range(last_course1_Fixed + 1, last_course1 + 1)
course12 = range(last_course1 + 1, last_course2_Fixed + 1)
course13 = range(last_course2_Fixed + 1, last_course2+ 1)
periods = range(num_periods)
weekdays = range(5)
# ------------------------------------------------------------------------------------------------

# Takes a screenshot
def getter(no):
    x=0
    y=100
    x1=3360
    y1=1900
    ImageGrab.grab().crop((x,y,x1,y1)).save("./scheds/sched" + str(no) + ".png")

# The following is the GUI that present the schedule graphically. 
# window is the GUI
# a widget is a component in a GUI
# The grid() method helps position the widgets into the grid configured into the GUI

# This function creates set up widget
def setupWidget(widget1, window):
    # List of strings of each weekdays (excluding Sunday)
    days_in_week = [ "Monday", "Tuesday", "Wednesday", "Thursday", "Friday","Saturday"]

    # List of strings of time (5 minutes increments starting from 8:00 to 16:30)
    times = []
    for i in range(8, 17):
        for j in range(6):
            for k in [0,5]:
                # if i == 8:
                #     if j < 3:
                #         continue
                if i == 16:
                    if j >3:
                        continue
                times.append(str(i) + ":" + str(j) + str(k))
    
    # widget1 is a list of widgets (used to keep track of widgets)
    # window is the GUI

    # set up row for days
    for i in range(5):
        # append a button to the widget list, with colors and text (e.g. "Monday")
        widget1.append(Button(window, text = days_in_week[i],fg ='black', bg = "grey"))
        # modify the last added widget to position in the first row (the row used for days).
        # Spans 4 columns. Each sub-column for each type of course slots  (50m, 75m, lab, and others)
        # Starts at the column that accounts for the first column used for times.
        # Sticky, so that the button expands to the border of its grid space
        widget1[-1].grid(row = 0, column=1 + i*4, sticky = 'nsew', columnspan= 4)

    # set up for column for times
    for i in range(0,len(times),3):
        # the first column holds the time (8:00, 8:15, 8:30, ..., 16:30)
        # first row is omitted since thats where the days of a week are held

        # Add a label to the widget with text (e.g. 8:00)
        widget1.append(ttk.Label(window, text = times[i]))
        # Modify the added widget to position at the next appropriate row space.
        # Row span is 3 and all widgets are positioned in the first column
        widget1[-1].grid(row = i+1, column=0, rowspan = 3)

# create widgets for courses of one of 4 types (50m, 75m, labs, others)
def assignWidget(widget1, window, day_period, offset, meeting_stat_map):

    # offset: what sub-column (under the day buttons) are the new widgets going to be positions
    # meeting_stat_map: a map of (k,v) entries for the number of a course slots of a specific course slots type (50m, 75m, 75m common hours, labs, 50m common hours, etc)
    # day_period: a 2D array of strings: "__" if no meeting is scheduled, slot number string if a meeting of that slot is scheduled
        # Each row is each day, each column is a start time. So [1,3] is 9:00 on Tuesday
        # The matrix hold information for meeting time of one of 4 types of course slots (50m, 75m, lab, others). So 4 matrixes will hold
        # the information for the whole schedule
    
    # variables to hold the number of rowspan for widgets of different course slots of different length (50m, 75m, 2hr, 3hr)
    onehr = int(50/5)
    one_half_hr = int(75/5)
    twohr = int(120/5)
    threehrs = int(180/5)

    # This dict will map a course slot to its type's label if it is scheduled (e.g. "1":"C50")
    outCourse = {}
    for ca in courses:
        outCourse[str(ca)] = ""

    # Holds the index of start times, and will make the keys in periodMap
    startPeriod = 0

    # The dict that maps to the start times (string) from their index values (e.g. 0:"8:00", 1:"8:30", 2:"9:00")
    periodMap ={}
    for i in range(8, 16):
        for j in ["00", "30"]:
            periodMap[startPeriod] = str(i) + ":" + j
            startPeriod = startPeriod + 1

    # map indexes to days of the week
    dayMap = {
        0: "M",
        1: "T",
        2: "W",
        3: "R",
        4: "F",
    }

    # will be used to map a course slot to its scheduled day (e.g. "8":"TR" for course 8 scheduled on Tuesday and Thursday)
    courseDayMap = {}
    # initialize
    for cd in courses:
        courseDayMap[str(cd)] = ""
    
    # will be used to map a course slot to its scheduled time (e.g. "0":"8:30" for course 8 scheduled at 8:30)
    coursePeriodMap = {}
    # initialize
    for cp in courses:
        coursePeriodMap[str(cp)] = ""

    # List of index for slots that can be scheduled at different times for different meetings (e.g. lab slot has 5 meetings that 
    # can be scheduled at different start times)
    flex_courses = list(course3)
    flex_courses.extend(course4)
    flex_courses.extend(course7)
    flex_courses.extend(course8)
    flex_courses.extend(course9)

    # A dict that maps a slot to its number of row span (e.g.: "0":"10" for button for course 0 spans 10 rows to cover row of 8:00 to row of 8:45)
    span = {}

    # A dict that maps a slot to its lab (e.g.: "10":"C75" for course 10 is of label C75 which indicates it is a 75m standard slot)
    labe = {}

    # Initialize values for span and labe
    for c in course1:
        span[str(c)] = onehr
        labe[str(c)] = "C50"
    for c in course2:
        span[str(c)] = one_half_hr
        labe[str(c)] = "C75"
    for c in course3:
        span[str(c)] = threehrs
        labe[str(c)] = "Lab"
    for c in course4:
        span[str(c)] = one_half_hr
        labe[str(c)] = "1.5hr common hr"
    for c in course5:
        span[str(c)] = onehr
        labe[str(c)] = "Lang"
    for c in course6:
        span[str(c)] = onehr
        labe[str(c)] = "Lang"
    for c in course7:
        span[str(c)] = onehr
        labe[str(c)] = "1h common hr"
    for c in course8:
        span[str(c)] = twohr
        labe[str(c)] = "Faculty Meeting (2hr)"
    for c in course9:
        span[str(c)] = threehrs
        labe[str(c)] = "Seminar"
    
    # Maps an index to a hex value of a color (e.g. "0":"#fe4a49")
    color = {
        "0" : "#fe4a49",
        "1" : "#651e3e",
        "2" : "#00b159",
        "3" : "#ff3377",
        "4" : "#55a61c",
        "5" : "#6497b1",
        "6" : "#3d1e6d",
        "7" : "#451e3e",
        "8" : "#0392cf",
        "9" : "#dbaa00",
        "10" : "#0054ff",
        "11" : "#fe8a71",
        "12" : "#0e9aa7",
        "13" : "#081f2d",
        "14" : "#d26939",
        "15" : "#195465",
        "16" : "#673888",
        "17" : "#D04848",
        "18" : "#89ce00",
        "19" : "#854442",
        "20" : "#fc6716",
        "21" : "#ff8948",
        "22" : "#9bd21d",
        "23" : "#ad89ff",
        "24" : "#6a329f",
        "25" : "#211C6A",
    }

    # List of courses that do not overlap with any other course slots at any of their meetings (e.g. 75m common hours slot)
    spanall = list(course4)
    spanall.extend(list(course7))
    spanall.extend(list(course8))

    # The following loop create widgets for meetings in day_period
    # for each index in day_period (representing whether a meeting is scheduled at a certain time)
    for r in weekdays:
        for p in periods:
            # if a meeting is scheduled
            if day_period[r,p] != " __ ":
                # take the value of the at that index for the course slot
                course = day_period[r,p]
                # change the value of the label for that slot
                outCourse[course] = labe[course]
                # modify the value of the days the meetings of the slot are scheduled
                courseDayMap[course] += dayMap[r]
                # modify the number of meetings of for all courses of the same type as this course
                meeting_stat_map[labe[course]] += 1

                # if the course slot is scheduled at different start times, modify the value of the time scheduled
                if (int(course) in flex_courses):
                    coursePeriodMap[course] += " " + periodMap[p]
                # else, set a new start time for the course slot
                else:
                    coursePeriodMap[course] = periodMap[p]

                # If a course slot does not overlap with any other courses, create a button with a label, color, and at the same row as its start time
                # Place the button at the first sub-column, rowspan as calculated in the span dict, and column span all 4 sub-columns. Sticky
                if int(course) in spanall:
                    widget1.append(Button(window, text = labe[course],fg ='white', bg=color[course]))
                    widget1[-1].grid(row = 1+p*6, column=1 + r*4 , sticky = 'nsew', rowspan= span[course], columnspan = 4)
                # Else, create a button with a label, color, and at the same row as its start time
                # Place the button at the appropriate sub-column (offset), rowspan as calculated in the span dict. Sticky
                else:
                    widget1.append(Button(window, text = labe[course] +": "+ course,fg ='white', bg=color[course]))
                    widget1[-1].grid(row = 1+p*6, column=1 + offset +r*4, sticky = 'nsew', rowspan= span[course])

    # Map each label to a header for text display
    headerMap = {
        "C50" : "50-minutes standard course slots: ",
        "C75" : "75-minutes standard course slots: ",
        "Lab" : "Lab slots: ",
        "Seminar" : "Language, seminar, and common hour slots: ",
        "1.5hr common hr" : "Language, seminar, and common hour slots: ",
        "Lang" : "Language, seminar, and common hour slots: ",
        "1h common hr": "Language, seminar, and common hour slots: ",
        "Faculty Meeting (2hr)": "Language, seminar, and common hour slots: ",
    }


    # Display the text (course label, course #, days scheduled, start time scheduled)
    header = True
    # Display the header of the course slots of one of 4 types
    for c1 in courses:
        if outCourse[str(c1)] != "":
            if header == True:
                print("----------------------" + headerMap[labe[str(c1)]] + "----------------------")
                header = False

    # Display the slots for one of 4 types
    not_flex_courses = list(courses)
    for fc in flex_courses:
        not_flex_courses.remove(fc)
    for p in periods:
        for c1 in not_flex_courses:
            if outCourse[str(c1)] != "":
                if (periodMap[p] == coursePeriodMap[str(c1)]):
                    print(labe[str(c1)] + " " + str(c1) + ": " + courseDayMap[str(c1)] + "  " + coursePeriodMap[str(c1)])

    for fc in flex_courses:
        if outCourse[str(fc)] != "":
            print(labe[str(fc)] + " " + str(fc) + ": " + courseDayMap[str(fc)] + "  " + coursePeriodMap[str(fc)])


# meeting_stat_map: a map of (k,v) entries for the number of a course slots of a specific course slots type (50m, 75m, 75m common hours, labs, 50m common hours, etc)
meeting_stat_map = {
        "C50": 0,
        "C75": 0,
        "Lab": 0,
        "1.5hr common hr": 0,
        "Lang" : 0,
        "1h common hr" : 0,
        "Faculty Meeting (2hr)" : 0,
        "Seminar" : 0,        

    }

def outputGUI(day_period1, day_period2, day_period3, day_period4, no):
    # Create a window
    window = tk.Tk()
    # window's title, background and size
    window.title('Grid')
    window['bg'] = 'white'
    window.geometry('1920x1080')

    # blank list to hold widgets
    widget1 = []


    # define a grid
    # Configure columns and uniformity
    window.columnconfigure(list(range(1+5*4)), weight=1, uniform='a')
    # Configure rows and uniformity
    window.rowconfigure(list(range(1+(num_periods+2)*6)), weight = 1, uniform = 'a')
    # Reconfigure the first row (days of week)
    window.rowconfigure(0, weight = 5, uniform = 'a')

    print("---------------------------------------------------------------------------------------------------------------")
    # Create set up widgets
    setupWidget(widget1, window)

    # Create widgets for courses of type 50m 
    assignWidget(widget1, window, day_period1, 0, meeting_stat_map)
    # Create widgets for courses of type 75m 
    assignWidget(widget1, window, day_period2, 1, meeting_stat_map)
    # Create widgets for courses of type lab 
    assignWidget(widget1, window, day_period3, 2, meeting_stat_map)
    # Create widgets for courses of type others
    assignWidget(widget1, window, day_period4, 3, meeting_stat_map)

    print()
    # Print out how many standard courses and common hours are scheduled
    print("----------------------" + "Statistics" + "----------------------")
    print("There are " + str(meeting_stat_map["C50"]//3) + " 50-minutes standard courses")
    print("There are " + str(meeting_stat_map["C75"]//2) + " 75-minutes standard courses")
    print("There are " + str(meeting_stat_map["1.5hr common hr"]) + " 1.5hr common hrs")
    print("There are " + str(meeting_stat_map["1h common hr"]) + " 1h common hrs")
    print("---------------------------------------------------------------------------------------------------------------")

    # only use if you want to automatically take a screenshot.
    # getter(no)

    #start the gui
    window.mainloop()





In [5]:
import numpy as np # numpy to create arrays (1D, 2D)
# guroby to create model
import gurobipy as gp 
from gurobipy import GRB

try:
    # Multiple optimal answers (1) or single optimal answer (0)
    switch = 1
    # Number of solutions if multiple optimal answer option is picked
    solutionNum = 20
    # Modify number of solutions in case the the single optimal answer is picked
    if switch == 0:
        solutionNum = 1
    # Create a new model
    m = gp.Model("cs25")

    # Terminologies: 
        # Period means the same as start time
        # Course means the same as class and course slots
        # Class meeting and course meeting means the same
        # 75-minute means the same as 1.5hr (because the start times has a 30-min increment)
        # 50-minte means that same as 1hr (because the start times has a 30-min increment)

    # Big M value
    M = 10^6
    # Number of days in a week to be scheduled
    num_days = 6
    #  Index set for days in the week {0,1,2,3,4,5} for Monday ... Saturday
    days = range(num_days)
    # last day of the week
    num_weekdays = 5
    #  Index set for school days {0,1,2,3,4} for Monday ... Friday
    weekdays = range(num_weekdays)
    
    # Last index for course1 index set (6 courses for course 1)
    last_course1 = 5
    # Last index for course10, course1 courses that do not overlap course2 courses (3 courses)
    last_course1_Fixed = 2
    # Last index for course2 index set (9 courses for course 2)
    last_course2 = 14     
    # Last index for course12, course2 courses that do not overlap course1 courses (3 courses)
    last_course2_Fixed = last_course1 + 3
    # Last index for lab index set 
    last_course3 = last_course2+1     
    # Last index for 1.5hr common hour index set
    last_course4 = last_course3+1 
    # Last index for morning language courses index set
    last_course5 = last_course4+3   
    # Last index for afternoon language courses index set
    last_course6 = last_course5+1
    # Last index for 1hr common hour index set
    last_course7 = last_course6+1
    # Last index for faculty meeting index set
    last_course8 = last_course7+1 
    # Last index for seminar index set
    last_course9 = last_course8+1
    # Last index for the set of all courses
    num_courses = last_course9 +1

    # Create index sets for courses based of the last indexes just established
    courses = range(num_courses) 
    course1 = range(last_course1 + 1)
    course2 = range(last_course1 + 1, last_course2 + 1)
    course3 = range(last_course2 +1 , last_course3 + 1)
    course4 = range(last_course3 +1, last_course4 + 1)
    course5 = range(last_course4 +1, last_course5 + 1)
    course6 = range(last_course5 + 1, last_course6 +1)
    course7 = range(last_course6 + 1, last_course7 +1)
    course8 = range(last_course7 + 1, last_course8 +1)
    course9 = range(last_course8 + 1, last_course9 +1)
    course10 = range(last_course1_Fixed + 1)
    course11 = range(last_course1_Fixed + 1, last_course1 + 1)
    course12 = range(last_course1 + 1, last_course2_Fixed + 1)
    course13 = range(last_course2_Fixed + 1, last_course2+ 1)

    # index set for days that can potential has early start time
    early_starts = range(num_weekdays)

    # total number of start times
    num_periods = 16
    # last index for afternoon start times
    last_aft_prd = 15
    # last index for morning start times  
    last_mor_prd = 7 

    # S
    periods = range(num_periods)
    # S1
    mor_prd = range(last_mor_prd + 1)
    # S2
    aft_prd = range(last_mor_prd + 1 ,last_aft_prd+1)
    # S3
    lab_periods = range(last_mor_prd+1,last_mor_prd+5)
    # S4
    seminar_periods = range(last_mor_prd+1,last_mor_prd+5)
    # S5
    one_and_half_hour_period = range(num_periods - 1)
    # S6
    faculty_period = range(last_aft_prd -2)
    # S7
    middle_periods = range(5,14)

    # Total number of decision variables x,w,a,e
    dec_var_num = num_days*num_courses*num_periods + num_courses*num_periods + num_weekdays + last_course2 + 1
    # first index for (a) decision variables (a course is scheduled if one of these variables is 1)
    reach_course_avail = num_days*num_courses*num_periods + num_courses*num_periods + num_weekdays
    # first index for (w) decision variables
    dpc_offset = num_days*num_courses*num_periods
    # first index for (e) decision variables
    dpcw_offset = num_days*num_courses*num_periods + num_courses*num_periods
    # offset value for a day's worth of x decision variables
    day_offset = num_courses*num_periods
    # offset value for a period's worth of x decision variables
    period_offset = num_courses

    # Special sets --------------------------------------------------------------------------------------
    # Course that meets five times a week
    five_prd_courses = list(course3)
    five_prd_courses.extend(list(course5))
    five_prd_courses.extend(list(course6))

    # periods that are not valid for falcuty meeting
    non_falc_periods = list(periods)
    for p in faculty_period:
        non_falc_periods.remove(p)

    # periods that are not valid for seminar meeting
    non_sem_periods = list(periods)
    for p in seminar_periods:
        non_sem_periods.remove(p)

    # Courses that meets for 75 minutes
    one_and_half_hr_courses = list(course2)
    one_and_half_hr_courses.extend(list(course4))

    # Courses that only start in the afternoon
    afternoon_courses = list(course3)
    afternoon_courses.extend(list(course6))

    # Start time that are invalid for labs
    non_lab_periods = list(periods)
    for l in lab_periods:
        non_lab_periods.remove(l)
    
    # Courses, each of which meets five times a week at the same start time
    five_prd_courses_fixed = five_prd_courses.copy()
    for c3 in course3:
        five_prd_courses_fixed.remove(c3)

    #Language courses
    lang_courses = list(course5)
    lang_courses.extend(list(course6))

    # Set of morning language courses and labs
    mlang_lab_courses = list(course5)
    mlang_lab_courses.extend(list(course3))

    # Courses/common hour that meets for 1 hour    
    one_hour_courses = list(course1)
    one_hour_courses.extend(list(course5))
    one_hour_courses.extend(list(course6))
    one_hour_courses.extend(list(course7))

    # Courses that meets for 2 hours
    two_hour_courses = list(course8)

    # Courses that meets for 3 hours
    three_hour_courses = list(course3)
    three_hour_courses.extend(course9)
    
    # courses that are not 1.5hr common hour
    not_one_and_half_comhr_courses = list(courses)
    for c4 in course4:
        not_one_and_half_comhr_courses.remove(c4)
    
    # Set of courses that meets for 1hr except for 1hr common hr
    one_hour_courses_exclude_common_hr = list(course1)
    one_hour_courses_exclude_common_hr.extend(list(course5))
    one_hour_courses_exclude_common_hr.extend(list(course6))

    # Courses that are not 1hr common hr course
    not_one_comhr_courses = list(courses)
    for c7 in course7:
        not_one_comhr_courses.remove(c7)

    # Courses that are not faculty meeting
    not_falc_courses = list(courses)
    for c8 in course8:
        not_falc_courses.remove(c8)

    # Courses that meets for 1hr excluding afternoon language class
    one_hour_courses_not_aft_lang = list(course1)
    one_hour_courses_not_aft_lang.extend(list(course5))
    one_hour_courses_not_aft_lang.extend(list(course7))

    # Courses that are neither seminar nor labs nor afternoon language course
    not_sem_lab_aft_lang_courses = list(courses)
    for c9 in course9:
        not_sem_lab_aft_lang_courses.remove(c9)
    for c3 in course3:
        not_sem_lab_aft_lang_courses.remove(c3) 
    for c6 in course6:
        not_sem_lab_aft_lang_courses.remove(c6)   

    # Flex courses: courses that have different start times for meetings
    flex_courses = list(course3)
    flex_courses.extend(course4)
    flex_courses.extend(course7)
    flex_courses.extend(course8)
    flex_courses.extend(course9)

    # Special sets --------------------------------------------------------------------------------------

    # Number of row in a matrix (2D array). This will tell how many additional constraints can be added
    default_mat_row = 8000

    # Add decision variables to the model (binary type). We add the coeficients later by using dot product with a vector 
    # of coeficients
    x = m.addMVar(shape=dec_var_num, vtype=GRB.BINARY, name="x")

    # Set objective coeficients
    
    # This is a vector for the coefficients of decision variables
    obj = np.zeros((dec_var_num))


    # Penalize if a meeting is scheduled before 9 AM
    for c in courses:
        for d in weekdays:
            for p in range(0,2):
                obj[d*day_offset + p*period_offset + c] += -100

    # Reward if a 1.5h common hour is scheduled
    for c4 in course4:
        for d in weekdays:
            for p in one_and_half_hour_period:
                obj[d*day_offset + p*period_offset + c4] += 20

    # Further reward if a 1.5h common hour is scheduled in the mid day
    for c4 in course4:
        for d in weekdays:
            for p in middle_periods:
                obj[d*day_offset + p*period_offset + c4] += 60
    
    # Reward if a 1h common hour is scheduled
    for c7 in course7:
        for d in weekdays:
            for p in periods:
                obj[d*day_offset + p*period_offset + c7] += 3 

    # Further reward if a 1h common hour is scheduled in the mid day
    for c7 in course7:
        for d in weekdays:
            for p in middle_periods:
                obj[d*day_offset + p*period_offset + c7] += 15


    # Reward if a 75-minute course slot is scheduled
    for c2 in course2:
        for d in weekdays:
            for p in periods:
        # obj[reach_course_avail + c] = 50
                obj[d*day_offset + p*period_offset + c2] += 130  

    # Reward if a 50-minute course slot is scheduled
    for c1 in course1:
        for d in weekdays:
            for p in periods:
        # obj[reach_course_avail + c] = 50
                obj[d*day_offset + p*period_offset + c1] += 10         

    # Reward if a lab slot can be scheduled early
    for c3 in course3:
        for d in weekdays:
            for p in range(7,10):
        # obj[reach_course_avail + c] = 50
                obj[d*day_offset + p*period_offset + c3] += 20    
    
    # # obj 3: faculty meeting
    # for c8 in course8:
    #     for d in weekdays:
    #         for p in faculty_period:
    #             obj[d*day_offset + p*period_offset + c8] = 1 

    # Set the objective function to maximize with the decision variables vector and the coefficients vector
    m.setObjective(obj @ x, GRB.MAXIMIZE)

    # Create the 2D array for constraints coefficients in the left hand side and initialize them to 0
    A = np.zeros((default_mat_row,dec_var_num))
    # List for the right hand side vector
    rhs = []
    # The index of the next constraint's left hand side coefficients
    cnst = 0

    # A[cnst, index] corresponds to the value for the coeficient of the decision variable x_correspondIndex of 
    # a specific constraint

    # Constraints are divided in to groups with are in turn divided into sections. For each constraint groups, we specify
    # each constraint's left hand side coefficient in a new row. The corresponding right hand side value will be added to
    # the right hand side vector.

    # The specific coefficients' indexes can be reached using offets and index values.

    # We divide the sections in 2 grand types: equal constraints and less than or equal constraints

    # (I) Number of meetings in specific courses ----------------------------------------------------------------------

    #const 1 coeficients =. 1hr standard meets three times per week
    for c1 in course1:
        for d in days:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
        
        A[cnst, reach_course_avail + c1] = -3
        cnst += 1
        rhs.append(0)
        
    #const 2 coeficients =. 1.5 hr standard class meets twice per week
    for c2 in course2:
        for d in days:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
        A[cnst, reach_course_avail + c2] = -2
        cnst += 1
        rhs.append(0)
    
    #const 3 coeficients = (5 periods for several classes)
    for c3 in five_prd_courses:
        for d in days:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c3] = 1
        cnst += 1
        rhs.append(5)

    # const 4 coeficients = one seminar meeting per week
    for c9 in course9:
        for d in weekdays:
            for p in seminar_periods:
                A[cnst, d*day_offset + p*period_offset + c9] = 1
    cnst+=1
    rhs.append(1)

    # (I) Number of meetings in specific courses ----------------------------------------------------------------------


    # (II) Invalid course periods ----------------------------------------------------------------------

    #const 5 coeficients = No classes on Saturday
    for p in periods:
        for c in courses:
            A[cnst,5*day_offset + p*period_offset + c] = 1
    cnst += 1   
    rhs.append(0)    

    # const 6 coeficients = a faculty meeting can only be scheduled in Faculty Meeting start times
    for p in non_falc_periods:
        for d in weekdays:
            for c8 in course8:
                A[cnst, d*day_offset + p*period_offset + c8] = 1
    cnst+= 1
    rhs.append(0)

    # const 7 coeficients = a seminar meeting can only be scheduled in seminar start times
    for p in non_sem_periods:
        for d in weekdays:
            for c9 in course9:
                A[cnst, d*day_offset + p*period_offset + c9] = 1
    cnst+= 1
    rhs.append(0)


    # const 8  coeficients = Start time 15 is not valid for 1.5hr courses
    for d in weekdays:
        for c2 in one_and_half_hr_courses:
            A[cnst, d*day_offset + (num_periods-1)*period_offset + c2] = 1
    cnst +=1
    rhs.append(0)
    
    # const 9 coeficients = no afternoon meetings for morning lang courses

    for c5 in course5:
        for d in weekdays:
            for ap in aft_prd:
                A[cnst, d*day_offset + ap*period_offset + c5] = 1
    cnst += 1
    rhs.append(0)

    # const 10 coefs = no morning meetings for labs and afternoon lang courses
    for ac in afternoon_courses:
        for d in weekdays:
            for mp in mor_prd:
                A[cnst, d*day_offset + mp*period_offset + ac] = 1
    cnst += 1
    rhs.append(0)

    #const 11 coefs = lab meetings should not be in the morning or late afternoons

    for nlp in non_lab_periods:
        for d in weekdays:
            for c3 in course3:
                A[cnst, d*day_offset + nlp*period_offset + c3] = 1
    cnst += 1
    rhs.append(0)

    
    
    # (II) Invalid course periods ----------------------------------------------------------------------



    # (III) Fixed meeting times ----------------------------------------------------------------------

    # const 12 coeficients =. All 1hr standard course will be schedule at the same start time
    for c1 in course1:
        for p in periods:
            for d in days:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
            A[cnst, dpc_offset + p*period_offset + c1] = -3
            cnst +=1
            rhs.append(0)

    # const 13 coeficients =. All 1.5hr standard course will be schedule at the same start time
    for c2 in course2:
        for p in periods:
            for d in days:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
            A[cnst, dpc_offset + p*period_offset + c2] = -2
            cnst +=1
            rhs.append(0)
    
    # const 14 coeficients. All language courses will be schedule at the same start time
    for fpcf in five_prd_courses_fixed:
        for p in periods:
            for d in days:
                A[cnst, d*day_offset + p*period_offset + fpcf] = 1
            A[cnst, dpc_offset + p*period_offset + fpcf] = -5
            cnst +=1
            rhs.append(0)
    
    # (III) Fixed meeting times ----------------------------------------------------------------------

    # delete unused matrix row
    A=np.delete(A, range(cnst,default_mat_row), axis=0) 

    # convert list of right hand side values into np form
    nprhs = np.array(rhs)

    # Add the constraints to the model by using matrix multiplication
    m.addConstr(A @ x == nprhs, name="ceq")  

    # Restart the process of add constraints coefficients into the matrix for less than or equal to constraints
    A = np.zeros((default_mat_row,dec_var_num))
    rhs = []
    cnst = 0

    # (IV) Standard course available ----------------------------------------------------------------------
    
    # const 15 coeficients <=. Schedule at least 11 standard course slots (1h 3-meetings/week, 1.5h 2-meetings/week)
    standard_courses = list(course1)
    standard_courses.extend(list(course2))
    for c in standard_courses:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-11)

    # const 16 coeficients <=. Schedule at least 3 1h standard course slots (1h 3-meetings/week)
    for c in course1:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-3)

    # const 17 coeficients <=. Schedule at least 2 1h standard course slots that dont overlap with
    # any 1.5h standard course slots (1h 3-meetings/week non-overlapping)
    for c in course10:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-2)

    # const 18 coeficients <=. Schedule at least 5 1.5h standard course slots (1.5h 2-meetings/week)
    for c in course2:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-5)

    # const 19 coeficients <=. Schedule at least 2 1.5h standard course slots that dont overlap with
    # any 1h standard course slots (1.5h 2-meetings/week non-overlapping)
    for c in course12:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-2)

    # const 20 coeficients <=. Schedule at least 2 1.5hr common hours
    for d in weekdays:
        for p in one_and_half_hour_period:
            for c in course4:
                A[cnst, d*day_offset + p*period_offset + c] =-1
    cnst += 1
    rhs.append(-2)
    # (IV) Standard course available ----------------------------------------------------------------------



    # (V) Early starts ----------------------------------------------------------------------
    # const ES (21) <=. At most eax (5) days can have early start out of the weekdays 
    eax = 5
    for ea in early_starts:
        A[cnst, dpcw_offset + ea] = 1
    cnst += 1
    rhs.append(5-eax)

    # const ES (22) <=. Only allowed days can have early start time out of the weekdays (lang courses automatically do not start early if 
    # at most 4 days are allowed to start early)
    for ea in early_starts:
        for c in courses:
            A[cnst, ea*day_offset + c] = 1
        A[cnst, dpcw_offset + ea] = -M
        cnst += 1
        rhs.append(0)
    # (V) Early starts ----------------------------------------------------------------------


    # (VI) Day spacing for each course ------------------------------------------------------------------------

    # const 23 coeficients <=. Spacing for 1hr standard course
    for c1 in course1:
        for d in weekdays:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
                A[cnst, ((d+1)%num_days)*day_offset + p*period_offset + c1] = 1
                #A[cnst, ((d+2)%num_days)*day_offset + p*period_offset + c1] = 1
            cnst += 1
            rhs.append(1)
            # rhs.append(1)


    # const 24 <=. coeficients <=. Day spacing for 1.5hr standard course
    for c2 in course2:
        for d in weekdays:
            for p in one_and_half_hour_period:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
                A[cnst, ((d+1)%(num_days-1))*day_offset + p*period_offset + c2] = 1
            cnst += 1
            rhs.append(1)

    # const 25 <=. Day spacing for 5 meetings courses
    for c2 in five_prd_courses:
        for d in weekdays:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
            cnst += 1
            rhs.append(1)

    # (VI) Day spacing for each course ------------------------------------------------------------------------



    # (VII) Collision of courses' meetings -------------------------------------------------------------------------

    # const 26 <= . 1hr standard courses do not overlap with each other
    exclude_last_periods = list(one_and_half_hour_period)
    for d in weekdays:
        for p in exclude_last_periods:
            for c1 in course1:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
                A[cnst, d*day_offset + (p+1)*period_offset + c1] = 1
            cnst += 1
            rhs.append(1)

    # const 27 <=. 1.5hr courses do not overlap with each other
    for d in weekdays:
        for p in range(num_periods-2):
            for coah in one_and_half_hr_courses:
                A[cnst, d*day_offset + p*period_offset + coah] = 1
                A[cnst, d*day_offset + (p+1)*period_offset + coah] = 1
                A[cnst, d*day_offset + (p+2)*period_offset + coah] = 1
            cnst += 1
            rhs.append(1)

    # const 28 <=. 1hr common hours do not overlap with each other
    for d in weekdays:
        for p in exclude_last_periods:
            for c7 in course7:
                A[cnst, d*day_offset + p*period_offset + c7] = 1
                A[cnst, d*day_offset + (p+1)*period_offset + c7] = 1
            cnst += 1
            rhs.append(1)

    #const 29 <=. Mor & Aft lang courses cannot overlap with each other. 
    for p in exclude_last_periods:
        for ml in lang_courses:
            A[cnst, p*period_offset + ml] = 1
            A[cnst, (p+1)*period_offset + ml] = 1
        cnst +=1
        rhs.append(1)
    
    # const 30 <=. (mlang + lab cant overlap with each other). Works at the moment because labs are only scheduled for the afternoon. 
    # In other words with the current model a lab always starts after morning language classes.
    for p in mor_prd:
        for mlc in mlang_lab_courses:
            A[cnst, p*period_offset + mlc] = 1
            A[cnst, (p+1)*period_offset + mlc] = 1
        cnst +=1
        rhs.append(1)


    # const 31 <=. non-colide for 1hr fixed courses with 1.5hr course
    for c1 in course10:
        for p in periods:
            for d in weekdays:
                for c2 in course2:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                    A[cnst, d*day_offset + p*period_offset + c2] = 1
                    if (p+1 <= num_periods):
                        A[cnst, d*day_offset + (p+1)*period_offset + c2] = 1
                    A[cnst, d*day_offset + p*period_offset + c1] = M 
                    cnst+=1
                    rhs.append(M)
    
    # const 32 <=. non-colide for 1hr courses with 1.5hr fixed course
    for c1 in course1:
        for p in periods:
            for d in weekdays:
                for c2 in course12:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                    A[cnst, d*day_offset + p*period_offset + c2] = 1
                    if (p+1 <= num_periods):
                        A[cnst, d*day_offset + (p+1)*period_offset + c2] = 1
                    A[cnst, d*day_offset + p*period_offset + c1] = M 
                    cnst+=1
                    rhs.append(M)

    # const 33 <=. overlapping patern for 1hr courses flex with 1.5hr course flex
    for c1 in course11:
        for p in periods:
            for d in weekdays:
                for c2 in course13:
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                    if (p+1 <= num_periods):
                        A[cnst, d*day_offset + (p+1)*period_offset + c2] = 1
                    A[cnst, d*day_offset + p*period_offset + c1] = M 
                    cnst+=1
                    rhs.append(M)

    # (VII) Collision of courses' meetings -------------------------------------------------------------------------

    
    # (VIII) Special arangement for courses ---------------------------------------------------------------------
    # const 34 coefs <= . no classes overlap with 1.5 common hours 
    for ch in course4:
        for d in days:
            for p in one_and_half_hour_period:
                for ohc in one_hour_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in course2:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for twohc in two_hour_courses:
                    for index in range(1,4):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + twohc] = 1
                for thc in three_hour_courses:
                    for index in range(1,6):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + thc] = 1
                for cl in not_one_and_half_comhr_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    A[cnst, d*day_offset + (p+1)*period_offset + cl] = 1 
                    if (p+2) <= last_aft_prd : 
                        A[cnst, d*day_offset + (p+2)*period_offset + cl] = 1 
                        
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)
    
    # const 35 coefs <=. no classes overlap with 1hr common hours 
    for ch in course7:
        for d in days:
            for p in periods:
                for ohc in one_hour_courses_exclude_common_hr:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in one_and_half_hr_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for twohc in two_hour_courses:
                    for index in range(1,4):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + twohc] = 1
                for thc in three_hour_courses:
                    for index in range(1,6):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + thc] = 1
                for cl in not_one_comhr_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    if (p+1) <= last_aft_prd : 
                        A[cnst, d*day_offset + (p+1)*period_offset + cl] = 1 
                        
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)

    # const 36 coefs <=. no classes overlap with 2h faculty meeting  
    for ch in course8:
        for d in days:
            for p in faculty_period:
                for ohc in one_hour_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in one_and_half_hr_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for thc in three_hour_courses:
                    for index in range(1,6):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + thc] = 1
                for cl in not_falc_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    for index in range(1,4):
                        if (p+index) <= num_periods-1 : 
                            A[cnst, d*day_offset + (p+index)*period_offset + cl] = 1 
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)


    # const 37 coefs <=. no classes overlap with seminar except for labs, and afternoon language course  
    for ch in course9:
        for d in days:
            for p in seminar_periods:
                for ohc in one_hour_courses_not_aft_lang:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in one_and_half_hr_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for cl in not_sem_lab_aft_lang_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    for index in range(1,6):
                        if (p+index) <= num_periods-1 : 
                            A[cnst, d*day_offset + (p+index)*period_offset + cl] = 1 
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)
    
    # (VIII) Special arangement for courses ---------------------------------------------------------------------

    # Delete unused rows
    A=np.delete(A, range(cnst,default_mat_row), axis=0) 

    # print the number of row and column of A in case debugging is needed
    print(np.shape(A))

    # convert right hand side values array to an approriate form
    nprhs = np.array(rhs)    

    # add less than or equal constraints to the model
    m.addConstr(A @ x <= nprhs, name="cleq")
    
    # Do the following if multiple solution option is chosen, else the solver will return a single optimal answer if found
    if switch == 1:
        # percentage away from the optimal solution
        m.setParam("poolGap", 0.05) 
        # ask for a specific number of solutions
        m.setParam("PoolSolutions",solutionNum)
        # search mode for multiple solutions
        m.setParam("PoolSearchMode", 1)
        # Time limit for the model in seconds
        m.setParam("TimeLimit", 300)
 # Optimize model
    m.optimize()
    
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

(5683, 2708)
Set parameter PoolGap to value 0.05
Set parameter PoolSolutions to value 20
Set parameter PoolSearchMode to value 1
Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6015 rows, 2708 columns and 63814 nonzeros
Model fingerprint: 0xadcc7a7d
Variable types: 0 continuous, 2708 integer (2708 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 4225 rows and 1701 columns
Presolve time: 0.31s
Presolved: 1790 rows, 1007 columns, 32590 nonzeros
Variable types: 0 continuous, 1007 integer (1007 binary)

Root relaxation: objective 3.037100e+03, 1455 iterations, 0.05 seconds (0.06 work units)

    Nodes    |    Curren

In [6]:
# print a 2D array for view
def print2D(day_period):
    for r in day_period:
        for c in r:
            if c == " __ ":
                print(c, end="")
            elif len(c) == 1:
                print("  "+c+" ", end="")
            else:
                print(" "+c+" ", end="")

        print("")


# decision variable array
ans = []

# step to iterate through solutions
step = int(solutionNum/5)

# if switch == 0 or <= 4, modify step to make sure the loop terminate
if switch == 0 or solutionNum < 5:
    step = 1

# iterate through the solutions
for index in range(0, solutionNum, step):
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Schedule " + str(index) + " >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    if switch == 1:
        # set solution number to obtain from the model
        m.setParam("solutionNumber", index)

        # get the solution, print it and its objective value
        ans = x.Xn
        print(ans)
        print('Obj: %g' % m.PoolObjVal)

    if switch ==0:
        # get the solution, print it and its objective value
        ans = x.X
        print(ans)
        print('Obj: %g' % m.ObjVal)
    
    # value to initiate 4 2D arrays
    empty = " __ "

    # This 2D array will hold the values of whether a meeting for a 1h standard course slots is scheduled
    # at day i, start time j. So for example day_period1[3,2] = "0" will mean that course slot 0 has a
    # a meeting at 9 AM on Thursday. Another example is day_period1[1,1] = "__" will mean that no meeting of any 
    # standard course slot is scheduled at 8:30 AM on Thursday 
    day_period1 = np.full((num_days, num_periods),empty)
    # Same as above, but this 2D array is for values of 1.5h standard course slots.
    day_period2 = np.full((num_days, num_periods),empty)
    # Same as above, but this 2D array is for values of lab slots.
    day_period3 = np.full((num_days, num_periods),empty)
    # Same as above, but this 2D array is for values of the remaining slots.
    day_period4 = np.full((num_days, num_periods),empty)

    # We need a total of 4 2D arrays because 2D arrays cant deal with too much overlapping

    # extract value from the solution to the 2D array for 1h standard course slots
    for c1 in course1:
        for d in days:
            for p in periods:
                # for all possible meeting date d and start time p in course1, if there is a meeting for
                # a course slot in course1, record the index of that course slot into the [d,p] position of the array
                if ans[d*day_offset + p*period_offset + c1] == 1:
                    if day_period1[d,p] == empty :
                        day_period1[d,p] = c1

    # extract value from the solution to the 2D array for 1.5h standard course slots
    for c2 in course2:
        for d in days:
            for p in periods:
                if ans[d*day_offset + p*period_offset + c2] == 1:
                    if day_period2[d,p] == empty :
                        day_period2[d,p] = c2
    
    # extract value from the solution to the 2D array for lab slots
    for c3 in course3:
        for d in days:
            for p in periods:
                if ans[d*day_offset + p*period_offset + c3] == 1:
                    if day_period3[d,p] == empty :
                        day_period3[d,p] = c3

    non_standardlab_courses = list(course4)
    non_standardlab_courses.extend(list(course5))
    non_standardlab_courses.extend(list(course6))
    non_standardlab_courses.extend(list(course7))
    non_standardlab_courses.extend(list(course8))
    non_standardlab_courses.extend(list(course9))

    # extract value from the solution to the 2D array for the remaining course slots. Notice that if
    # the value for the seminar slot is not shown, you can check in the graphical representation and 
    # notice if there is an afternoon that does not have any standard course in it, then the seminar
    # has the same start time as the afternoon language meeting that day. 
    for nsc in non_standardlab_courses:
        for d in days:
            for p in periods:
                if ans[d*day_offset + p*period_offset + nsc] == 1:
                    if day_period4[d,p] == empty :
                        day_period4[d,p] = nsc
    
    # print the text of the 2D arrays out
    print("-----------------------------------------------------------")
    print2D(day_period1)
    print("-------------------------------------------------------------")
    print2D(day_period2)
    print("-------------------------------------------------------------")
    print2D(day_period3)
    print("-------------------------------------------------------------")
    print2D(day_period4)
    print("-------------------------------------------------------------")
    
    # graphical representation
    outputGUI(day_period1, day_period2, day_period3, day_period4, index)
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print()
    print()
    

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Schedule 0 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[0. 0. 0. ... 1. 1. 1.]
Obj: 2310
-----------------------------------------------------------
 __  __   4  __   1  __   0  __  __  __  __  __   3  __   5  __ 
 __  __  __  __  __  __  __  __  __  __  __  __  __  __  __  __ 
 __  __   4  __   1  __   0  __  __  __  __  __   3  __   5  __ 
 __  __  __  __  __  __  __  __  __  __  __  __  __  __  __  __ 
 __  __   4  __   1  __   0  __  __  __  __  __   3  __   5  __ 
 __  __  __  __  __  __  __  __  __  __  __  __  __  __  __  __ 
-------------------------------------------------------------
 __  13  __  __  __  __  __  __  10  __  __  14  __  __  11  __ 
 __  __   7  __  __   6  __  __  __  __  __   9  __  __  12  __ 
 __  13  __  __  __  __  __  __  10  __  __  14  __  __  11  __ 
 __  __   7  __  __   6  __  __  __  __  __  __  __  __  __  __ 
 __  __  __  __  __  __  __  __  __  __  __   9  __  __  12  __ 
 __  __  __  __  __  __  _